In [1]:
from MALBuddy import MALBuddy
import pandas as pd
import os
import re

client_info_filepath = "client_info.json"
token_filepath = "token.json"

In [2]:
buddy = MALBuddy(client_info_filepath, token_filepath)

In [3]:
user = 'chocopieeater'
anime_list = buddy.get_anime_list(user)
anime_list.head()

,id,title,main_picture,status,score,num_episodes_watched,is_rewatching,updated_at,finish_date,start_date
0,41457,86,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,0,False,2021-05-02T00:36:51+00:00,NaN,NaN
1,22199,Akame ga Kill!,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,8,24,False,2019-02-26T07:05:44+00:00,NaN,NaN
2,41433,Akudama Drive,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,7,False,2020-12-28T21:14:38+00:00,NaN,NaN
3,22729,Aldnoah.Zero,{'medium': 'https://api-cdn.myanimelist.net/im...,completed,6,12,False,2018-08-07T02:05:12+00:00,NaN,NaN
4,27655,Aldnoah.Zero 2nd Season,{'medium': 'https://api-cdn.myanimelist.net/im...,dropped,0,11,False,2018-08-07T01:31:22+00:00,NaN,NaN


In [4]:
title_dict = anime_list[['title','id']].set_index('title')
title_dict = title_dict['id'].to_dict()
titles = anime_list['title']

In [5]:
title_dict

{'86': 41457,
 'Akame ga Kill!': 22199,
 'Akudama Drive': 41433,
 'Aldnoah.Zero': 22729,
 'Aldnoah.Zero 2nd Season': 27655,
 'Angel Beats!': 6547,
 'Ansatsu Kyoushitsu': 24833,
 'Ansatsu Kyoushitsu 2nd Season': 30654,
 'Ao no Exorcist': 9919,
 'Assassins Pride': 38572,
 'Bakemonogatari': 5081,
 'Black Bullet': 20787,
 'Black Clover': 34572,
 'Blade & Soul': 22547,
 'Blend S': 34618,
 'Boku dake ga Inai Machi': 31043,
 'Boku no Hero Academia': 31964,
 'Boku no Hero Academia 2nd Season': 33486,
 'Boku no Hero Academia 3rd Season': 36456,
 'Boku no Hero Academia 4th Season': 38408,
 'Boku no Hero Academia 5th Season': 41587,
 "Boku no Hero Academia the Movie 3: World Heroes' Mission": 44200,
 'Chainsaw Man': 44511,
 'Chuunibyou demo Koi ga Shitai!': 14741,
 'Cowboy Bebop': 1,
 'Darling in the FranXX': 35849,
 'Death March kara Hajimaru Isekai Kyousoukyoku': 34497,
 'Death Note': 1535,
 'Dororo': 37520,
 'Dr. Stone': 38691,
 'Dr. Stone: Stone Wars': 40852,
 'Dungeon ni Deai wo Motomeru no 

In [6]:
def format_title(title: str) -> str:
    title = re.sub(' ', '_', title.lower())
    title = re.sub('\W', '', title)
    return title

In [7]:
anime = 'Shingeki no Kyojin'
anime

'Shingeki no Kyojin'

In [8]:
z = buddy.generate_and_write_ratings(format_title(anime), title_dict[anime], num_pages=5)

Successfully loaded pages: : 1,2,3,4,5,
Ratings saved to anime_ratings\shingeki_no_kyojin.json 


In [9]:
a = buddy.load_ratings(format_title(anime))

anime_ratings\shingeki_no_kyojin.json


In [10]:
a

,index,user,rating
0,0,laurachiaragrit2,8
1,1,Salvation01,8
2,2,Cadiz95,10
3,3,Remilia12,7
4,4,dimgoku,10
...,...,...,...
434,208,FionaCG,8
435,209,taytay1625,8
436,210,Joacoalma,9
437,211,Sul1260,5


In [12]:
(a['user'] == 'mohamed_arafa38').sum()

1

In [12]:
a

,user,rating
0,mohamed_arafa38,7
1,ZARAGAMBA,10
2,Aotfanboi,10
3,5atru1ne,9
4,gabilparra,10
...,...,...
222,hunchsos,10
223,buttfungys,8
224,TMOJ,8
225,GianSenpai,10


In [14]:
b = buddy.load_ratings(format_title(anime))

anime_ratings\shingeki_no_kyojin.json


In [17]:
b

,user,rating
0,mohamed_arafa38,7
1,ZARAGAMBA,10
2,Aotfanboi,10
3,5atru1ne,9
4,gabilparra,10
...,...,...
222,hunchsos,10
223,buttfungys,8
224,TMOJ,8
225,GianSenpai,10


In [88]:
b['user']

,user,rating
0,biyumekodir,8
1,robonaldo,6
2,CEOofSex,6
3,Hero1A,10
4,qilanime,10
...,...,...
180,NatsumeSv,9
181,BongaBonga21,6
182,Tetonet,8
183,XELA_blakrose,9


In [95]:
b.set_index('user')['SwagyJuztin']

,rating
user,
biyumekodir,8
robonaldo,6
CEOofSex,6
Hero1A,10
qilanime,10
...,...
NatsumeSv,9
BongaBonga21,6
Tetonet,8


In [12]:
#TODO allow writing over and updating json files when generating new ratings from MAL

In [13]:
save_folder = 'anime_ratings'
fp = os.path.join(save_folder, f'{title}.json')
fp

'anime_ratings\\akame_ga_kill.json'

In [14]:
os.path.exists(fp)

True

In [28]:
import json

In [38]:
with open(fp, 'r') as file:
    ratings2 = json.load(file)

In [60]:
r1 = pd.DataFrame(ratings)

In [61]:
r2= pd.DataFrame(ratings2)

In [64]:
r1

,user,rating
0,xAsiy,10
1,NikyNN,8
2,Jig0ki,8
3,certifiedfreak,7
4,xiaoguang,7
...,...,...
134,z_xd,8
135,Chriszcamp,9
136,GraceSage,8
137,Ayyor,7


In [69]:
pd.concat([r1,r2])

,user,rating
0,xAsiy,10
1,NikyNN,8
2,Jig0ki,8
3,certifiedfreak,7
4,xiaoguang,7
...,...,...
155,NatsumeSv,9
156,BongaBonga21,6
157,Tetonet,8
158,XELA_blakrose,9
